In [1]:
# Descrição
# Este notebook conta o número de registros de cada arquivo em cada etapa de processamento.
#
# Description
# This notebook counts the number of registers in each file in each processing step.

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1607391432872_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Spark Config
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing necessary packages for notebook session
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Running setup.py bdist_wheel for boto3: started
  Running setup.py bdist_wheel for boto3: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/d7/49/8d/c7b244e2b89911f7eb4d8db3b2ec5700b443e280fa68b6f9a0
Successfully built boto3

In [3]:
from dateutil import parser
# lib boto3 to use aws services
import boto3

# counting file sizes aka number of records
counters = {"MO_1510" + str(day): {"raw_data": 0, "hour-6-23-no-duplicates": 0, "hour-6-23-in-sp": 0} for day in range(1,32)}


for key in counters.keys():
    # reading raw data
    traces_raw = spark.read.csv("s3a://mobility-traces-sp/raw-data/" + key + ".csv")
    # count raw data records
    counters[key]["raw_data"] = traces_raw.count()
    
    # reading records_between_6-23_hour-no-duplicates
    traces_hour_6_23 = spark.read.parquet("s3a://mobility-traces-sp/processed-data-avl-date/1-records-between-6-23-avl-hour-dropped-columns-no-duplicates/" + key + "/")
    # counting records
    counters[key]["hour-6-23"] = traces_hour_6_23.count()
    
    # reading records_between_6_23_only_in_sp
    traces_hour_6_23_in_sp = spark.read.parquet("s3a://mobility-traces-sp/processed-data-avl-date/3-records-in-sp-between-6-23/" + key + "/")
    
    # counting records
    counters[key]["hour-6-23-in-sp"] = traces_hour_6_23_in_sp.count()

# file header
csv_out = "day,raw_data_count,hour_6_23_count,hour_6_23_in_sp_count\n"
for key in counters.keys():
    # writing csv text
    csv_out += key +","+str(counters[key]["raw_data"])+","+str(counters[key]["hour-6-23"])+","+str(counters[key]["hour-6-23-in-sp"])+"\n"
    
s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/2-number-records-per-file-after-processing.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'B3A46F2AB9CD2AC5', 'HostId': 'ViWu0Ha2k5iU8Vb1RSaWl2zvQMkfCsvB2wghJpawXHiT8ohXSm4u9b6UMDh4cUsZBH73L2WWL+0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'ViWu0Ha2k5iU8Vb1RSaWl2zvQMkfCsvB2wghJpawXHiT8ohXSm4u9b6UMDh4cUsZBH73L2WWL+0=', 'x-amz-request-id': 'B3A46F2AB9CD2AC5', 'date': 'Tue, 08 Dec 2020 02:21:09 GMT', 'etag': '"b1762362e853bfed8270d0d79b975cc6"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"b1762362e853bfed8270d0d79b975cc6"'}

In [ ]:
# executed in a cluster with 1 master m5.xlarge, 4 m5.4xlarge, 2 nodes m5.xlarge
# each round took in average 22/25seconds to run